# Model adaptive phase space reconstruction

In [ ]:
%matplotlib notebook
import os
import time
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

import schemes as sc
import dynamicalsystems as dyns

# Define parameters

In [ ]:
class model_param:
    def __init__(self):
        self.method = sc.rk4   # Method of integration
        self.batch_time=20     # Time duration of training batch in terms of Δt steps
        self.batch_size=300    # Number of batches of training data
        self.lr = 1e-6         # Learning rate for model parameters
        self.lr_τ = 1e-6       # Leaning rate delay vector τ
        self.Nc = 200          # Maximum delay in terms of Δt steps
        self.niters=20000      # Maximum number of iterations
        self.test_freq=100     # Testing frequency in terms of iterations (for print and plot)
        self.viz=True          # Visualization
        self.savefig= True     # Set True to save figure
        self.folder='output_Lorenz_neural'   # folder name to store output
        self.gpu=0             # GPU to be used (not yet implemented)

In [ ]:
args = model_param()

# Define device

In [ ]:
device = torch.device('cuda:' + str(args.gpu) if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')   # Remove once code is compatible with gpu 

# Folder to save output

In [ ]:
try:
    os.mkdir(args.folder)
except:
    response = input("Warning: Folder alrady exist. Figures will be replaced. Do you want to replace? (y/n)")
    if response.lower()!='y':
        args.savefig= False

# Generate Time Series

In [ ]:
t_true = torch.linspace(0, 120, 12001)
x_data = torch.tensor([1,1,0]).reshape(1,-1) # Initial Condition
param  = [10., 28., 8./3.] # [σ, ρ, β]

dt  = t_true[1] - t_true[0]
fun = lambda t,x: dyns.Lorenz(x,param)
for i in range(1, len(t_true)):
    x_data = torch.cat([x_data, sc.rk4(fun, t_true, x_data[i-1,:], dt).reshape(1,-1)], 0)
    
fig = plt.figure()
axs = fig.add_subplot(1, 1, 1)
label = ['x','y','z']
[axs.plot(t_true, x_data[:,i], label = label[i]) for i in range(x_data.shape[1])]
axs.set_xlim([min(t_true), max(t_true)])
axs.set_xlabel('t')
plt.legend()
plt.show()

# Normalized time series

In [ ]:
N_ts = 10000
t_true = t_true[0:N_ts]
x_data_sam   = x_data[-len(t_true):,:]

##################################
x_data = x_data_sam- x_data_sam.mean(0)
x_data = x_data/x_data.abs().max(0).values
##################################

fig = plt.figure()
axs = fig.add_subplot(1, 1, 1)

label = ['x','y','z']
[axs.plot(t_true, x_data[:,i], label = label[i]) for i in range(x_data.shape[1])]
axs.set_xlim([min(t_true), max(t_true)])
axs.set_xlabel('t')
plt.legend()
plt.show()

if x_data.shape[1]==3:
    fig = plt.figure()
    axs = fig.add_subplot(projection='3d')
    axs.plot(x_data[:,0], x_data[:,1], x_data[:,2])
    plt.show()

# Observed data

In [ ]:
x_true = x_data[:,0]
t_true = t_true

# Time step

In [ ]:
dt = t_true[1] - t_true[0]

# Method to creates batch_size number of batches of true data of batch_time duration

In [ ]:
def get_batch(t, x, Nc, τ, batch_time, batch_size):
    dt = t[1]-t[0]
    assert τ.max()<Nc*dt, "Maximum value of delay should be less than Nc*dt="+str(Nc*dt)+'.'
    z_true = sc.interp_linear(t, x, Nc, τ)
    id_sel = torch.randint(0, z_true.shape[0] - batch_time-1, (batch_size,))
    z_true_stack = torch.stack([z_true[id_sel + i, :] for i in range(batch_time)], dim=0)
    t_true_stack = torch.stack([t_true[id_sel + i] for i in range(batch_time)], dim=0)
    return t_true_stack.to(device), z_true_stack.to(device)

# Integrate the <i>fun</i> from initial state $z_0$ for given time array $t$

In [ ]:
def get_pred(fun, z0, t):
    dt = len(t)
    z_pred = z0
    for i in range(1, len(t)):
        z_next = args.method(fun, t[i], z_pred[i - 1, :, :], dt)
        z_pred = torch.cat([z_pred, z_next.reshape(1, z_pred.shape[1], z_pred.shape[2])], 0)
    return z_pred

# Define ODE (To avoid changes in program at other location degree is specified inside the class)

In [ ]:
class ODEFunc(nn.Module):
    
    def __init__(self, n):
        super(ODEFunc, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(n, n+50),
            nn.Tanh(),
            nn.Linear(n+50, n),
        )

        for m in self.net.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, mean=0, std=0.001)
                nn.init.constant_(m.bias, val=0)

    def forward(self, t, y):
        return self.net(y)  

# Function to modify delay vector and return modified ODE function 

In [ ]:
def get_fun(func, optimizer, optimizer_τ, lr, lr_τ, τ, acc):
    l_0 = len(τ)
    τ = sc.merge(τ, acc)
    
    if l_0>len(τ):
        print('Merged τ:',τ)
        func = ODEFunc(n=len(τ))
        
    τ = torch.tensor(τ.detach().numpy(), device=device, requires_grad=True)
    optimizer = optim.RMSprop(func.parameters(), lr=lr)   # lr=1e-4
    optimizer_τ = optim.RMSprop([τ], lr=lr_τ)             # lr=1e-6 => Gives ok results
    
    return func, optimizer, optimizer_τ, τ

In [ ]:
def plot_cmp(fig, title, kk, tt,zt, tp,zp):
    plt.clf()
    fig.set_size_inches(6, (zt.shape[2]-1)*8, forward=True)
    axs_arr = [fig.add_subplot(zt.shape[2]-1,1,i+1) for i in range(zt.shape[2]-1)] 
    for p_id in range(args.batch_size):
        for i in range(zt.shape[2]-1):
            axs_arr[i].plot(zt[:, p_id, 0].detach().numpy(), zt[:, p_id, i+1].detach().numpy(),'k-')
            axs_arr[i].plot(zp[:, p_id, 0].detach().numpy(), zp[:, p_id, i+1].detach().numpy(),'r--', linewidth=2)
            axs_arr[i].set_title(title)
            axs_arr[i].set_xlabel('x(t)',fontsize=20)
            axs_arr[i].set_ylabel('x(t+$\\tau_'+str(i+1)+'$)',fontsize=20)
            plt.tight_layout()
        
    plt.show(block=False)
    fig.canvas.draw()
    
    if args.savefig:
        fig_name = args.folder+'/'+str(kk)+'.png'
        print('Saving figure:', fig_name)
        plt.savefig(fig_name)
    plt.pause(0.001)

# Initialize delay vector $\tau$ and $acc$. If $|\tau_i-\tau_j|<acc$ then $\tau_i$ and $\tau_j$ will merge

In [ ]:
τ = torch.tensor((torch.linspace(0,20,4)*dt).detach().numpy(), device=device, dtype=torch.float, requires_grad=True)
acc = 0.5*dt

# Initialize function and optimizer

In [ ]:
func = ODEFunc(n=len(τ))
optimizer = optim.RMSprop(func.parameters(), lr=args.lr)
optimizer_τ = optim.RMSprop([τ], lr=args.lr_τ)

# Main function

In [ ]:
fig = plt.figure(figsize=[6,8])

for kk in range(args.niters):
    func, optimizer, optimizer_τ, τ = get_fun(func, optimizer, optimizer_τ, args.lr, args.lr_τ, τ, acc)
    
    optimizer.zero_grad()
    optimizer_τ.zero_grad()
    
    t_batch, z_batch = get_batch(t_true, x_true, args.Nc, τ, args.batch_time, args.batch_size)
    z_pred = get_pred(func, z_batch[0,:,:].reshape(1, z_batch.shape[1], z_batch.shape[2]), t_batch[:,0])
    loss = torch.mean(torch.abs(z_pred - z_batch))
    loss.backward()
    #print('loss:',loss)
    optimizer.step()
    
    if kk>300:
        optimizer_τ.step()
        with torch.no_grad():
            τ[0] = 0.0
        
    if kk%args.test_freq==0:
        print('Iter:',kk)
        print('τ:',τ)
        #print('w0:',func.parameters())
        plot_cmp(fig, 'Iter:'+ str(kk)+', '+'$\\tau$='+str(τ.detach().numpy()), kk, t_batch, z_batch, t_batch, z_pred)
    